# Neural Network With pytorch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

* nn.Conv2d(input_channel, output_channel, square convolutional kernel
* nn.Linear(input_dimension, output_dimension)

In [97]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()    # 32x32   || 28x28 
        self.conv1 = nn.Conv2d(1, 6, 3)    # 30 x 30--> pool --> 15x15 || 26x26 --> pool --> 13x13
        # Formula for next layer ==> dimension - kernel + 1 / Stride
        self.conv2 = nn.Conv2d(6, 16, 3)   # 13x13 --> pool --> 6x6 || 
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self, x):
        # Maxing Over 2x2 window size
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2) )
        # Single 2 is equal to 2,2 . which means if it is square you can just specify one dimention
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
    
        return x
        
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Learnable Parameters are in net.parameters()

In [98]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [99]:
inpt = torch.randn(1, 1, 32, 32)
out = net(inpt)
print(out)

tensor([[0.0000, 0.0693, 0.0489, 0.1167, 0.0571, 0.0567, 0.1270, 0.0449, 0.0000,
         0.0782]], grad_fn=<ReluBackward0>)


In [100]:
net.zero_grad()
out.backward(torch.randn(1, 10))
print(out)

tensor([[0.0000, 0.0693, 0.0489, 0.1167, 0.0571, 0.0567, 0.1270, 0.0449, 0.0000,
         0.0782]], grad_fn=<ReluBackward0>)


In [101]:
output = net(inpt)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
print(target.size())
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

torch.Size([1, 10])
tensor(0.7549, grad_fn=<MseLossBackward>)


In [102]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0])

In [103]:
print(net.conv1.bias)

Parameter containing:
tensor([0.1573, 0.2877, 0.2886, 0.2178, 0.1003, 0.1720], requires_grad=True)


In [104]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([ 0.0210,  0.0304,  0.0018,  0.0374,  0.0238, -0.0140])
conv1.bias.grad after backward
tensor([ 0.0241,  0.0206, -0.0033,  0.0425,  0.0250, -0.0046])


In [105]:
learning_rate = 0.01

In [106]:
for f in net.parameters():
    f.data.sub_(learning_rate* f.grad.data)

In [107]:
import torch.optim as optim


In [108]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [109]:
optimizer.zero_grad()
output = net(inpt)
loss = criterion(output, target)
print(loss)
loss.backward()
optimizer.step()  

tensor(0.7561, grad_fn=<MseLossBackward>)
